# Project Setup

In [1]:
from __future__ import print_function
from __future__ import absolute_import

import glob
import librosa
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import time
from IPython import display
import requests
from io import BytesIO
import tensorflow as tf
import moviepy.video.io.ImageSequenceClip
from moviepy.editor import VideoFileClip, AudioFileClip

import tensorflow_hub as hub
tf.__version__

2023-01-19 00:19:07.631299: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-19 00:19:07.787490: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-19 00:19:07.793061: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-19 00:19:07.793076: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

'2.11.0'

In [2]:
# ADD YAMNET PACKAGE TO THE SYSTEM PATHS
import sys
absolute_path_yamnet = '/home/claudio-unix/mae/ShapingMusic/yamnet_pkg'
sys.path.insert(1, absolute_path_yamnet)

In [ ]:
# UTILS
def load_img(path_to_img):
    max_dim = 512
    img = tf.io.read_file(path_to_img)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)

    shape = tf.cast(tf.shape(img)[:-1], tf.float32)
    long_dim = max(shape)
    scale = max_dim / long_dim

    new_shape = tf.cast(shape * scale, tf.int32)

    img = tf.image.resize(img, new_shape)
    img = img[tf.newaxis, :]
    return img

def imshow(image, title=None):
    if len(image.shape) > 3:
        image = tf.squeeze(image, axis=0)
        plt.imshow(image)
    if title:
        plt.title(title)

def split_audio(file_name):
    audio, FS = librosa.load(file_name)
    Ts = 0.5
    hop_size = int(Ts*FS)
    N = int(len(audio)/hop_size)
    audio_list = [ audio[i*hop_size:(i+1)*hop_size] for i in range(N)]
    return audio_list

def retrieve_frames(path_dir):
    frames_list = []
    for filename in os.listdir(path_dir):
        path = os.path.join(path_dir, filename)
        # path = os.path.join(os.getcwd(), path)
        frames_list.append(path)
    return frames_list

def create_clip(images_path, audio_path):
    generated_frames_list = []
    for filename in os.listdir(images_path):
        path = os.path.join(images_path, filename)
        # path = os.path.join(os.getcwd(), path)
        generated_frames_list.append(path)

    # response = requests.get(image_url)
    # image = Image.open(BytesIO(response.content))
    # clip = moviepy.video.io.ImageSequenceClip.ImageSequenceClip(generated_frames_list, fps)
    # audio_clip = moviepy.audio.io.AudioFileClip.AudioFileClip(audio_filename)
    
    # clip.set_audio(audio_clip);
    # clip.write_videofile(video_filename)
    # np_image = np.array(image)

    fps = 2
    audio_filename = audio_path
    video_filename = 'data/generated_clip/Test.mp4'

    clip = moviepy.video.io.ImageSequenceClip.ImageSequenceClip(generated_frames_list, fps)
    clip.write_videofile(video_filename)

    audio_clip = AudioFileClip(audio_filename)
    # audio_clip = audio_clip.subclip(0, 25)

    video_clip = VideoFileClip(video_filename);
    video_clip = video_clip.set_audio(audio_clip);

    video_clip.write_videofile(video_filename)

In [3]:
# IMAGES_FEATURES_EXTRACTOR_MODEL
image_layers = [
    'block1_conv1',
    'block2_conv1',
    'block3_conv1', 
    'block4_conv1', 
    'block5_conv1'
    ]

image_model = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
image_model.trainable = False
outputs = [image_model.get_layer(name).output for name in image_layers]
images_FEM = tf.keras.Model(image_model.input, outputs)

In [3]:
# AUDIO MODEL
from yamnet_pkg import inference
audio_model = inference.YamNet()

2023-01-19 00:19:15.331861: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/claudio-unix/mae/ShapingMusic/myenv/lib/python3.8/site-packages/cv2/../../lib64:
2023-01-19 00:19:15.331905: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-19 00:19:15.331938: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (popcorn): /proc/driver/nvidia/version does not exist
2023-01-19 00:19:15.332282: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild

In [7]:
audio_layers = [
    'layer1/conv',
    'layer2/pointwise_conv',
    'layer3/pointwise_conv',
    'layer4/pointwise_conv',
    'layer5/pointwise_conv',
    'layer6/pointwise_conv',
    'layer7/pointwise_conv',
    'layer8/pointwise_conv',
    'layer9/pointwise_conv',
    'layer10/pointwise_conv',
    'layer12/pointwise_conv',
    'layer13/pointwise_conv',
    'layer14/pointwise_conv',
]
audio_model.trainable = False
outputs = [audio_model.get_layer(name).output for name in audio_layers]
audios_FEM = tf.keras.Model(audio_model.input, outputs)
audios_FEM.summary()

AttributeError: 'NoneType' object has no attribute 'output'

In [ ]:
# TEXT-TO-IMAGE GENERATOR
import yaml
import openai
with open("apikey.local.yml", "r") as stream:
    try:
        openai.api_key = yaml.safe_load(stream)['apikey']
    except yaml.YAMLError as exc:
        print(exc)
        print('Cannot load the APIKey')

In [ ]:
import videotest

class Audio_Image_Classifier:
    '''
    Class responsible for multimodal classification.
    Given an audio track and the corresponding sequence of frames to classify, produces a string by concatenating labels.
    sound_model: pretrained tf.Model for audio classification
    image_model: pretrained tf.Model for image classification
    '''

    def __init__(self, image_model, sound_model):
        self.image_model = image_model
        self.sound_model = sound_model
        self.frame_count = 0
        self.fps = 2
        self.frames_path = 'data/test_frames'
        self.sounds_path = 'data/test_samples/'
        self.full_audios_path = 'data/test_audio'
        self.generated_frames_path = 'data/generated_frames/'
        self.generated_clips_path = 'data/generated_clips/'


    def __call__(self, video_url):
        
        if(video_url.endswith('.mp4') == False):
            return
        self.filename = video_url[0:-4]

        self.frame_count = 0
        self.__preprocess_video(video_url)

        img_list, snd_list = self.__retrieve_data()
        
        for i in range(len(img_list)):
            self.__classify(img_list[i], snd_list[i])
        print('Completed! Generating clip...')
        
        self.__generate_clip()
        
        self.__cleanup()
        
        print('DONE!')


    def __preprocess_video(self, video_url):
        videotest.main(video_url)
    
    
    def __retrieve_data(self):
        frames_list = []
        for filename in os.listdir(self.frames_path):
            path = os.path.join(self.frames_path, filename)
            frames_list.append(path)

        sounds_list = []
        for filename in os.listdir(self.sounds_path):
            path = os.path.join(self.sounds_path, filename)
            frames_list.append(path)
            
        return (frames_list, sounds_list)


    def __classify(self, img, snd):

        # Image Classification
        img = load_img(img)
        input_img = tf.keras.applications.vgg19.preprocess_input(img * 255)
        input_img = tf.image.resize(input_img, (224, 224))
        predictions_img = self.image_model(input_img)
        predictions_img = tf.keras.applications.vgg19.decode_predictions(predictions_img.numpy())[0]

        # Sound Classification
        predictions_snd = sound_model(snd)

        # Clip Generation
        text = self.__parse_predictions(predictions_img, predictions_snd)
        try:
            self.__generate_image(text)
        except:
            try: 
                print(text)
                print('Your prompt may contain text that is not allowed by our safety system.')
                text = predictions_snd
                self.__generate_image(text)
            except:
                try:
                    print(text)
                    print('Your prompt may contain text that is not allowed by our safety system.')
                    text = self.__parse_predictions(predictions_img, [])
                    self.__generate_image(text)
                except:
                    print(text)
                    print('Your prompt may contain text that is not allowed by our safety system.')
                    self.__generate_image(predictions_snd[0])


    def __parse_predictions(self, predictions_img, predictions_snd):
        predictions_img = [class_name for (number, class_name, prob) in predictions_img]
        predictions_img = [class_name.split('_') for class_name in predictions_img]
        input_text = predictions_snd
        for words in predictions_img:
            for w in words:
                input_text.append(w)
        return ' '.join(input_text)


    def __generate_image(self, text):
        response = openai.Image.create(
            prompt = text,
            n=1,
            size="512x512"
        )
        image_url = response['data'][0]['url']
        response = requests.get(image_url)
        img = PIL.Image.open(BytesIO(response.content))
        self.__store_image(img)


    def __store_image(self, img):
        generated_frame_path = 'img_at_frame_{}.png'.format(self.frame_count)
        path = os.path.join(self.generated_frames_path, generated_frame_path)
        tf.keras.utils.save_img(path, img)
        self.frame_count += 1


    def __generate_clip(self):
        output_file_path = os.path.join(self.generated_clips_path, 'Test.mp4')
        
        generated_frames_list = []
        for filename in os.listdir(self.images_path):
            path = os.path.join(self.images_path, filename)
            generated_frames_list.append(path)

        clip = moviepy.video.io.ImageSequenceClip.ImageSequenceClip(generated_frames_list, self.fps)
        clip.write_videofile(output_file_path)

        audio_clip = AudioFileClip(self.audio_path)
        video_clip = VideoFileClip(output_file_path).set_audio(audio_clip)
        video_clip.write_videofile(output_file_path)
    

    def __cleanup(self):
        return

In [ ]:
classifier = Audio_Image_Classifier(image_model, sound_model)

In [ ]:
classifier('data/test_video/test1.mp4')